## Import Library & API

In [53]:
import pandas as pd
import pandas_ta as ta
from pandas.tseries.offsets import BusinessDay
import pandas_market_calendars as mcal
import pywt
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.graph_objects as go
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.ardl import ARDL, ardl_select_order, UECM
from statsmodels.stats.diagnostic import het_arch
from statsmodels.tools.sm_exceptions import ValueWarning
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
import fredapi as fa
from datetime import date
from twelvedata import TDClient
import vectorbt as vbt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Input, LSTM, BatchNormalization, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.losses import Huber
import time
import re
import os

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore', ValueWarning)

os.chdir('/Users/fulinq/Documents/KMITL/FinancialEngineering/Y4/Y4T1/PROJECT/ARDL-ECM/Code/Gold/Finalized')

In [54]:
fred = fa.Fred(api_key='c948956426006ca126a2dd3bd1f07cee')
td = TDClient(apikey='aa61c51218c248698467af34d09b9d46')

## Data Retrieve ##

In [3]:
def fetch_fred(fred_client, series_id, col_name, percent = False, save_csv=False):
    df = fred_client.get_series(series_id)
    df.index = pd.to_datetime(df.index) 
    print(f'NaN value before processing: {df.isna().sum()}')
    df = df.ffill()
    print(f'NaN value after processing: {df.isna().sum()}')
    df.rename(col_name, inplace=True)
    print(f'Total records for {col_name}: {len(df)}')
    print(f'start date: {df.index.min()}')
    print(f'end date: {df.index.max()}')
    
    if percent:
        df = df.mul(0.01)
        print(f'Total records for {col_name} in percent: {len(df)}')
    
    if save_csv:
        filename = f"all_{col_name.lower()}_data_fred.csv"
        df.to_csv(filename)
        print(f"FRED data saved to {filename}")
    
    return pd.DataFrame(df)

def chow_lin_disaggregate(y_low: pd.Series, X_high: pd.DataFrame,
                          agg_method: str = 'sum', rho: float = None) -> tuple:
    y_low = y_low.dropna().copy()
    X_high = X_high.dropna().copy()
    n_high_per_low = 3  # Quarterly -> Monthly = 3 เดือนต่อไตรมาส

    # หาช่วงเวลาที่ซ้อนทับกัน (Overlapping period)
    quarters = y_low.index
    months = X_high.index
    min_date = max(quarters.min(), months.min().to_period('Q').to_timestamp())
    max_date = min(quarters.max(), months.max().to_period('Q').to_timestamp())

    y_low = y_low[(y_low.index >= min_date) & (y_low.index <= max_date)]
    
    # ปรับช่วงเวลาของ Monthly ให้ครอบคลุม Quarterly พอดี
    month_start = y_low.index.min()
    month_end = (y_low.index.max() + pd.offsets.QuarterEnd()).to_period('M').to_timestamp()
    X_high = X_high[(X_high.index >= month_start) & (X_high.index <= month_end)]

    n_low = len(y_low)
    n_high = n_low * n_high_per_low
    X_high = X_high.iloc[:n_high] # ตัดส่วนเกินออก

    # Build aggregation matrix C (Matrix สำหรับแปลงรายเดือนกลับเป็นไตรมาส)
    C = np.zeros((n_low, n_high))
    for i in range(n_low):
        start_col = i * n_high_per_low
        end_col = start_col + n_high_per_low
        if agg_method == 'sum': # สำหรับ Flow variable เช่น GDP
            C[i, start_col:end_col] = 1.0
        elif agg_method == 'mean': # สำหรับ Stock variable
            C[i, start_col:end_col] = 1.0 / n_high_per_low
        else:
            C[i, end_col - 1] = 1.0

    # Prepare X matrix
    X = X_high.values
    if X.ndim == 1: X = X.reshape(-1, 1)
    X = np.column_stack([np.ones(n_high), X]) # เพิ่ม Intercept

    # OLS เบื้องต้นเพื่อหาค่า Rho (Autocorrelation coefficient)
    X_low = C @ X
    y = y_low.values.flatten()
    beta_ols = np.linalg.lstsq(X_low, y, rcond=None)[0]
    u_low = y - X_low @ beta_ols

    if rho is None: # ถ้าไม่ได้กำหนดมา ให้คำนวณจาก Residuals
        if len(u_low) > 1:
            rho = np.corrcoef(u_low[:-1], u_low[1:])[0, 1]
            rho = np.clip(rho, -0.99, 0.99)
        else:
            rho = 0.0

    # GLS Estimation (พระเอกของงาน)
    # สร้าง Covariance Matrix V ตามโครงสร้าง AR(1)
    V = np.zeros((n_high, n_high))
    for i in range(n_high):
        for j in range(n_high):
            V[i, j] = rho ** abs(i - j)

    V_low = C @ V @ C.T
    try:
        V_low_inv = np.linalg.inv(V_low)
    except:
        V_low_inv = np.linalg.pinv(V_low)

    # คำนวณ Beta ด้วย GLS
    XVX = X_low.T @ V_low_inv @ X_low
    XVy = X_low.T @ V_low_inv @ y
    try:
        beta_gls = np.linalg.solve(XVX, XVy)
    except:
        beta_gls = np.linalg.lstsq(XVX, XVy, rcond=None)[0]

    # คำนวณค่าพยากรณ์และกระจาย Error (Distribute residuals)
    p_high = X @ beta_gls
    u_low_gls = y - X_low @ beta_gls
    VCt = V @ C.T
    
    try:
        dist_matrix = VCt @ np.linalg.inv(V_low)
    except:
        dist_matrix = VCt @ np.linalg.pinv(V_low)

    y_high = p_high + dist_matrix @ u_low_gls # ผลลัพธ์สุดท้าย

    result = pd.Series(y_high, index=X_high.index, name='GDP_Monthly_ChowLin')
    return result, beta_gls, rho

In [4]:
gold = vbt.YFData.download("GC=F", start="2006-01-01", interval="1d").get()
gold = pd.DataFrame(gold)
gold.columns = gold.columns.str.lower()
gold.index = pd.to_datetime(gold.index).tz_localize(None)
gold = gold.sort_index()
gold = gold.drop(columns=['dividends', 'stock splits'])
gold.to_csv('all_gold_data.csv')
gold = gold.get('close')
gold

Date
2006-01-03 05:00:00     530.700012
2006-01-04 05:00:00     533.900024
2006-01-05 05:00:00     526.299988
2006-01-06 05:00:00     539.700012
2006-01-09 05:00:00     549.099976
                          ...     
2026-02-12 05:00:00    4923.700195
2026-02-13 05:00:00    5022.000000
2026-02-17 05:00:00    4882.899902
2026-02-18 05:00:00    4986.500000
2026-02-19 05:00:00    5035.299805
Name: close, Length: 5063, dtype: float64

In [5]:
dollar_index = fetch_fred(fred, series_id='DTWEXBGS', col_name='Dollar Index')
dollar_index

NaN value before processing: 207
NaN value after processing: 0
Total records for Dollar Index: 5250
start date: 2006-01-02 00:00:00
end date: 2026-02-13 00:00:00


,Dollar Index
2006-01-02,101.4155
2006-01-03,100.7558
2006-01-04,100.2288
2006-01-05,100.2992
2006-01-06,100.0241
...,...
2026-02-09,117.6392
2026-02-10,117.5216
2026-02-11,117.4601
2026-02-12,117.5376


In [6]:
ppi = fetch_fred(fred, series_id='PPIACO', col_name='PPI')
ppi

NaN value before processing: 0
NaN value after processing: 0
Total records for PPI: 1356
start date: 1913-01-01 00:00:00
end date: 2025-12-01 00:00:00


,PPI
1913-01-01,12.100
1913-02-01,12.000
1913-03-01,12.000
1913-04-01,12.000
1913-05-01,11.900
...,...
2025-08-01,262.110
2025-09-01,262.094
2025-10-01,260.724
2025-11-01,261.358


In [7]:
fed_fund = fetch_fred(fred, series_id='FEDFUNDS', col_name='Federal Fund Rate', percent=True)
fed_fund

NaN value before processing: 0
NaN value after processing: 0
Total records for Federal Fund Rate: 859
start date: 1954-07-01 00:00:00
end date: 2026-01-01 00:00:00
Total records for Federal Fund Rate in percent: 859


,Federal Fund Rate
1954-07-01,0.0080
1954-08-01,0.0122
1954-09-01,0.0107
1954-10-01,0.0085
1954-11-01,0.0083
...,...
2025-09-01,0.0422
2025-10-01,0.0409
2025-11-01,0.0388
2025-12-01,0.0372


In [8]:
vix = fetch_fred(fred, series_id='VIXCLS', percent=True,col_name='VIX')
vix['VIX'] = vix['VIX'].mul(1 / np.sqrt(252))
vix

NaN value before processing: 301
NaN value after processing: 0
Total records for VIX: 9426
start date: 1990-01-02 00:00:00
end date: 2026-02-17 00:00:00
Total records for VIX in percent: 9426


,VIX
1990-01-02,0.010860
1990-01-03,0.011459
1990-01-04,0.012107
1990-01-05,0.012668
1990-01-08,0.012763
...,...
2026-02-11,0.011118
2026-02-12,0.013115
2026-02-13,0.012977
2026-02-16,0.013355


In [9]:
unemploy = fetch_fred(fred, series_id='ICSA', col_name='ISCA') #Initial Claims
unemploy

NaN value before processing: 0
NaN value after processing: 0
Total records for ISCA: 3084
start date: 1967-01-07 00:00:00
end date: 2026-02-07 00:00:00


,ISCA
1967-01-07,208000.0
1967-01-14,207000.0
1967-01-21,217000.0
1967-01-28,204000.0
1967-02-04,216000.0
...,...
2026-01-10,199000.0
2026-01-17,210000.0
2026-01-24,209000.0
2026-01-31,232000.0


In [10]:
ip = fetch_fred(fred, series_id='INDPRO', col_name='IP')
ip

NaN value before processing: 0
NaN value after processing: 0
Total records for IP: 1285
start date: 1919-01-01 00:00:00
end date: 2026-01-01 00:00:00


,IP
1919-01-01,4.8739
1919-02-01,4.6585
1919-03-01,4.5238
1919-04-01,4.6046
1919-05-01,4.6315
...,...
2025-09-01,101.7059
2025-10-01,101.2570
2025-11-01,101.3775
2025-12-01,101.6296


In [11]:
gdp = fetch_fred(fred, series_id='GDP', col_name='GDP')
gdp

NaN value before processing: 4
NaN value after processing: 4
Total records for GDP: 319
start date: 1946-01-01 00:00:00
end date: 2025-07-01 00:00:00


,GDP
1946-01-01,NaN
1946-04-01,NaN
1946-07-01,NaN
1946-10-01,NaN
1947-01-01,243.164
...,...
2024-07-01,29511.664
2024-10-01,29825.182
2025-01-01,30042.113
2025-04-01,30485.729


In [12]:
y_target = gdp['GDP']
X_indicator = ip[['IP']]

gdp_monthly_gls, beta, rho = chow_lin_disaggregate(y_low=y_target, X_high=X_indicator, agg_method='sum', rho=None)
print("Estimated Rho (Autocorrelation):", rho)
gdp = gdp_monthly_gls.copy()
gdp_monthly_gls

Estimated Rho (Autocorrelation): 0.99


1947-01-01       77.774623
1947-02-01       80.692609
1947-03-01       84.696768
1947-04-01       78.954071
1947-05-01       82.894270
                  ...     
2025-05-01    10144.515129
2025-06-01    10243.720934
2025-07-01    10350.652186
2025-08-01    10374.011335
2025-09-01    10373.363479
Name: GDP_Monthly_ChowLin, Length: 945, dtype: float64

In [13]:
fed_balance = fetch_fred(fred, series_id='WALCL', col_name='Fed Balance Sheet') #Federal Reserve Total Assets
fed_balance

NaN value before processing: 0
NaN value after processing: 0
Total records for Fed Balance Sheet: 1209
start date: 2002-12-18 00:00:00
end date: 2026-02-11 00:00:00


,Fed Balance Sheet
2002-12-18,719542.0
2002-12-25,732059.0
2003-01-01,730994.0
2003-01-08,723762.0
2003-01-15,720074.0
...,...
2026-01-14,6581700.0
2026-01-21,6584580.0
2026-01-28,6587568.0
2026-02-04,6605909.0


In [14]:
# 1. organize data
realtime_data = {
    'gold': gold,
    'dollar_index': dollar_index,
    'vix': vix,
    'fed_rate': fed_fund,
    'fed_balance': fed_balance,
    'labor_claims': unemploy
}

lagged_data = {
    'ip': ip,
    'gdp': gdp,
    'ppi': ppi
}

# 2. resample & rename
monthly_dfs = []

# process real-time
for name, data in realtime_data.items():
    # FIX: force rename for both Series and DataFrame to match the key (lowercase)
    if isinstance(data, pd.DataFrame):
        data = data.iloc[:, 0].to_frame(name)
    else:
        data = data.to_frame(name)
    
    if name in ['labor_claims', 'vix']:
        monthly_dfs.append(data.resample('ME').mean())
    else:
        monthly_dfs.append(data.resample('ME').last())

# process lagged
for name, data in lagged_data.items():
    if isinstance(data, pd.DataFrame):
        data = data.iloc[:, 0].to_frame(name)
    else:
        data = data.to_frame(name)
    monthly_dfs.append(data.resample('ME').last())

# 3. merge
df_final = pd.concat(monthly_dfs, axis=1)

# 4. handle lag (shift)
vars_to_shift = ['ip', 'ppi']
for col in vars_to_shift:
    df_final[col] = df_final[col].shift(1)
df_final['gdp'] = df_final['gdp'].shift(4)

# 5. target variable
df_final['target_gold'] = df_final['gold'].shift(-1)

# 6. feature selection
features = [
    'gold', 'dollar_index', 'vix', 'fed_rate', 
    'fed_balance', 'labor_claims', 
    'ip', 'gdp','ppi'
]

df_model = df_final[features + ['target_gold']].dropna()

# check
print(f"data range: {df_model.index.min().date()} to {df_model.index.max().date()}")
print(df_model.columns)
df_model

data range: 2006-01-31 to 2026-01-31
Index(['gold', 'dollar_index', 'vix', 'fed_rate', 'fed_balance',
       'labor_claims', 'ip', 'gdp', 'ppi', 'target_gold'],
      dtype='object')


,gold,dollar_index,vix,fed_rate,fed_balance,labor_claims,ip,gdp,ppi,target_gold
2006-01-31,570.799988,99.4311,0.007560,0.0429,828901.0,295750.0,98.0452,4330.641421,163.000,561.599976
2006-02-28,561.599976,99.7695,0.007842,0.0449,840555.0,290750.0,98.1999,4387.341134,164.300,581.799988
2006-03-31,581.799988,100.5600,0.007366,0.0459,833675.0,301750.0,98.2413,4449.366137,161.800,651.799988
2006-04-30,651.799988,98.1412,0.007480,0.0479,844572.0,303600.0,98.4628,4487.496729,162.200,642.500000
2006-05-31,642.500000,97.7705,0.009100,0.0494,851580.0,332750.0,98.7618,4515.040605,164.300,613.500000
...,...,...,...,...,...,...,...,...,...,...
2025-09-30,3840.800049,120.1368,0.009946,0.0422,6608395.0,234750.0,101.6247,10144.515129,262.110,3982.199951
2025-10-31,3982.199951,121.3859,0.011393,0.0409,6587034.0,226750.0,101.7059,10243.720934,262.094,4218.299805
2025-11-30,4218.299805,121.0527,0.012454,0.0388,6552419.0,217600.0,101.2570,10350.652186,260.724,4325.600098
2025-12-31,4325.600098,119.7456,0.009738,0.0372,6640618.0,219000.0,101.3775,10374.011335,261.358,4713.899902


In [15]:
df_model.to_csv('gold_price_model_data.csv')

In [16]:
df_ret = pd.DataFrame()
cols_to_transform = ['gold', 'gdp', 'ip', 'ppi','dollar_index', 'labor_claims', 'fed_balance'] # ไม่เอา IP, PPI ตามแผน Core Model
cols_not_to_transform = ['fed_rate', 'vix'] # ตัวแปรที่ไม่ทำ log return
for col in cols_to_transform:
    if col in df_model.columns:
        df_ret[f'{col}_ret'] = np.log(df_model[col]).diff()
for col in cols_not_to_transform:
    if col in df_model.columns:
        df_ret[f'{col}_change'] = df_model[col].diff()
    
df_ret.dropna(inplace=True)
df_ret

,gold_ret,gdp_ret,ip_ret,ppi_ret,dollar_index_ret,labor_claims_ret,fed_balance_ret,fed_rate_change,vix_change
2006-02-28,-0.016249,0.013008,0.001577,0.007944,0.003398,-0.017051,0.013962,0.0020,0.000282
2006-03-31,0.035337,0.014038,0.000422,-0.015333,0.007892,0.037135,-0.008219,0.0010,-0.000475
2006-04-30,0.113611,0.008533,0.002252,0.002469,-0.024347,0.006112,0.012986,0.0020,0.000113
2006-05-31,-0.014371,0.006119,0.003032,0.012864,-0.003784,0.091680,0.008263,0.0015,0.001620
2006-06-30,-0.046187,0.003576,0.000254,0.009088,0.004875,-0.085442,-0.008424,0.0005,0.001558
...,...,...,...,...,...,...,...,...,...
2025-09-30,0.100460,0.004646,-0.002646,-0.000946,-0.000594,0.020442,0.000759,-0.0011,0.000025
2025-10-31,0.036154,0.009732,0.000799,-0.000061,0.010344,-0.034673,-0.003238,-0.0013,0.001447
2025-11-30,0.057598,0.010385,-0.004423,-0.005241,-0.002749,-0.041190,-0.005269,-0.0021,0.001060
2025-12-31,0.025119,0.002254,0.001189,0.002429,-0.010856,0.006413,0.013371,-0.0016,-0.002716


## Data Preparation ##

In [17]:
df_model = pd.read_csv('gold_price_model_data.csv', index_col=0, parse_dates=True)
df_model

,gold,dollar_index,vix,fed_rate,fed_balance,labor_claims,ip,gdp,ppi,target_gold
2006-01-31,570.799988,99.4311,0.007560,0.0429,828901.0,295750.0,98.0452,4330.641421,163.000,561.599976
2006-02-28,561.599976,99.7695,0.007842,0.0449,840555.0,290750.0,98.1999,4387.341134,164.300,581.799988
2006-03-31,581.799988,100.5600,0.007366,0.0459,833675.0,301750.0,98.2413,4449.366137,161.800,651.799988
2006-04-30,651.799988,98.1412,0.007480,0.0479,844572.0,303600.0,98.4628,4487.496729,162.200,642.500000
2006-05-31,642.500000,97.7705,0.009100,0.0494,851580.0,332750.0,98.7618,4515.040605,164.300,613.500000
...,...,...,...,...,...,...,...,...,...,...
2025-09-30,3840.800049,120.1368,0.009946,0.0422,6608395.0,234750.0,101.6247,10144.515129,262.110,3982.199951
2025-10-31,3982.199951,121.3859,0.011393,0.0409,6587034.0,226750.0,101.7059,10243.720934,262.094,4218.299805
2025-11-30,4218.299805,121.0527,0.012454,0.0388,6552419.0,217600.0,101.2570,10350.652186,260.724,4325.600098
2025-12-31,4325.600098,119.7456,0.009738,0.0372,6640618.0,219000.0,101.3775,10374.011335,261.358,4713.899902


In [18]:
vars_to_log = ['gold', 'dollar_index', 'fed_balance', 'labor_claims', 'ip', 'gdp','ppi', 'target_gold']
for col in vars_to_log:
    df_model[f'ln_{col}'] = np.log(df_model[col])

model_vars = ['fed_rate', 'vix'] + [f'ln_{c}' for c in vars_to_log]
df_ardl = df_model[model_vars].dropna()

df_ardl

,fed_rate,vix,ln_gold,ln_dollar_index,ln_fed_balance,ln_labor_claims,ln_ip,ln_gdp,ln_ppi,ln_target_gold
2006-01-31,0.0429,0.007560,6.347039,4.599465,13.627856,12.597270,4.585429,8.373471,5.093750,6.330790
2006-02-28,0.0449,0.007842,6.330790,4.602863,13.641818,12.580219,4.587005,8.386479,5.101694,6.366127
2006-03-31,0.0459,0.007366,6.366127,4.610755,13.633599,12.617354,4.587427,8.400517,5.086361,6.479738
2006-04-30,0.0479,0.007480,6.479738,4.586407,13.646585,12.623466,4.589679,8.409050,5.088830,6.465367
2006-05-31,0.0494,0.009100,6.465367,4.582623,13.654849,12.715147,4.592711,8.415169,5.101694,6.419180
...,...,...,...,...,...,...,...,...,...,...
2025-09-30,0.0422,0.009946,8.253436,4.788631,15.703851,12.366276,4.621287,9.224688,5.568764,8.289590
2025-10-31,0.0409,0.011393,8.289590,4.798975,15.700614,12.331603,4.622085,9.234420,5.568703,8.347187
2025-11-30,0.0388,0.012454,8.347187,4.796226,15.695345,12.290414,4.617662,9.244805,5.563462,8.372306
2025-12-31,0.0372,0.009738,8.372306,4.785369,15.708716,12.296827,4.618851,9.247059,5.565891,8.458271


In [19]:
def run_adf_test(series, name):
    # Test at Level
    result = adfuller(series.dropna())
    p_value = result[1]
    
    if p_value <= 0.05:
        return f"I(0) - Stationary (p={p_value:.4f})"
    else:
        # ถ้า Level ไม่นิ่ง ให้ลอง Test แบบ Diff (First Difference)
        diff_result = adfuller(series.diff().dropna())
        diff_p_value = diff_result[1]
        
        if diff_p_value <= 0.05:
            return f"I(1) - Stationary at Diff (p={diff_p_value:.4f})"
        else:
            return f"I(2) or Higher (Non-Stationary) (p={diff_p_value:.4f})"
        
summary_data = []
for col in df_ardl.columns:
    status = run_adf_test(df_ardl[col], col)
    summary_data.append({'Variable': col, 'Status': status})

df_status = pd.DataFrame(summary_data)
df_status

,Variable,Status
0,fed_rate,I(1) - Stationary at Diff (p=0.0050)
1,vix,I(0) - Stationary (p=0.0008)
2,ln_gold,I(1) - Stationary at Diff (p=0.0000)
3,ln_dollar_index,I(1) - Stationary at Diff (p=0.0000)
4,ln_fed_balance,I(1) - Stationary at Diff (p=0.0000)
5,ln_labor_claims,I(0) - Stationary (p=0.0182)
6,ln_ip,I(1) - Stationary at Diff (p=0.0000)
7,ln_gdp,I(1) - Stationary at Diff (p=0.0000)
8,ln_ppi,I(1) - Stationary at Diff (p=0.0000)
9,ln_target_gold,I(1) - Stationary at Diff (p=0.0000)


In [20]:
X_cols = ['fed_rate'
          ,'ln_gold'
          ,'ln_dollar_index'
          ,'vix'
          ,'ln_labor_claims'
          ,'ln_ip'
        #   ,'ln_gdp'
          ,'ln_ppi'
        #   ,'ln_fed_balance'
          ]

X = df_ardl[X_cols].dropna()
X = sm.add_constant(X)

vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]

vif_data

,Variable,VIF
0,const,29112.077053
1,fed_rate,1.445183
2,ln_gold,5.108960
3,ln_dollar_index,1.824137
4,vix,1.743549
5,ln_labor_claims,2.576206
6,ln_ip,1.997837
7,ln_ppi,6.169819


In [21]:
train_date_str = '2015-12-31'
df_test_ardl = df_ardl[df_ardl.index <= train_date_str].copy()
df_test_ardl

,fed_rate,vix,ln_gold,ln_dollar_index,ln_fed_balance,ln_labor_claims,ln_ip,ln_gdp,ln_ppi,ln_target_gold
2006-01-31,0.0429,0.007560,6.347039,4.599465,13.627856,12.597270,4.585429,8.373471,5.093750,6.330790
2006-02-28,0.0449,0.007842,6.330790,4.602863,13.641818,12.580219,4.587005,8.386479,5.101694,6.366127
2006-03-31,0.0459,0.007366,6.366127,4.610755,13.633599,12.617354,4.587427,8.400517,5.086361,6.479738
2006-04-30,0.0479,0.007480,6.479738,4.586407,13.646585,12.623466,4.589679,8.409050,5.088830,6.465367
2006-05-31,0.0494,0.009100,6.465367,4.582623,13.654849,12.715147,4.592711,8.415169,5.101694,6.419180
...,...,...,...,...,...,...,...,...,...,...
2015-08-31,0.0014,0.012239,7.031388,4.705577,15.314040,12.525253,4.617324,8.713265,5.267343,7.017058
2015-09-30,0.0014,0.015454,7.017058,4.712509,15.316051,12.504324,4.615484,8.715785,5.256974,7.040098
2015-10-31,0.0012,0.010576,7.040098,4.702369,15.317216,12.495004,4.612689,8.715764,5.242276,6.971481
2015-11-30,0.0012,0.010181,6.971481,4.721530,15.314483,12.500606,4.607842,8.721109,5.233779,6.966307


In [22]:
y_col = 'ln_gold'
X_cols = ['fed_rate'
          ,'ln_dollar_index'
          ,'vix'
          ,'ln_labor_claims'
          ,'ln_ip'
        #   ,'ln_gdp'
          ,'ln_ppi'
        #   ,'ln_fed_balance'
          ]

data_ardl = df_test_ardl[[y_col] + X_cols].dropna()

custom_max_order = {
    'fed_rate': 6,
    'ln_dollar_index': 3,
    'vix': 3,
    'ln_labor_claims': 6,
    'ln_ip': 3,
    'ln_ppi': 3
}

sel_res = ardl_select_order(
    data_ardl[y_col], 
    maxlag=6, 
    exog=data_ardl[X_cols], 
    maxorder=custom_max_order,
    ic='aic'
)

print(f"Best AR Lags: {sel_res.ar_lags}")
print(f"Best DL Orders: {sel_res.dl_lags}")

Best AR Lags: [1, 2, 3, 4, 5]
Best DL Orders: {'fed_rate': [0, 1, 2, 3, 4, 5, 6], 'ln_dollar_index': [0, 1], 'vix': [0], 'ln_labor_claims': [0, 1, 2, 3, 4, 5, 6], 'ln_ip': [0, 1], 'ln_ppi': [0, 1, 2]}


In [23]:
ar_lag = max(sel_res.ar_lags) if isinstance(sel_res.ar_lags, list) else sel_res.ar_lags
dl_lags = {k: (max(v) if isinstance(v, list) else v) for k, v in sel_res.dl_lags.items()}
exog_order = {}
for i in dl_lags:
    exog_order[i] = max(1, dl_lags[i])
    
print(f"\n--- 2. ARDL Levels Analysis & Bounds Test ---")
model_ardl = ARDL(
    data_ardl[y_col], 
    lags=ar_lag, 
    exog=data_ardl[X_cols], 
    order=exog_order
)
res_ardl = model_ardl.fit()
res_ardl.summary()


--- 2. ARDL Levels Analysis & Bounds Test ---


<class 'statsmodels.iolib.summary.Summary'>
"""
                                  ARDL Model Results                                 
=====================================================================================
Dep. Variable:                       ln_gold   No. Observations:                  120
Model:             ARDL(5, 6, 1, 1, 6, 1, 2)   Log Likelihood                 217.207
Method:                      Conditional MLE   S.D. of innovations              0.037
Date:                       Thu, 19 Feb 2026   AIC                           -374.413
Time:                               14:49:26   BIC                           -292.065
Sample:                           07-31-2006   HQIC                          -340.989
                                - 12-31-2015                                         
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 15.5486      5.839      2.663      0.009       3.942      27.155
ln_gold.L1             0.6079      0.100      6.058      0.000       0.408       0.807
ln_gold.L2             0.1556      0.106      1.475      0.144      -0.054       0.365
ln_gold.L3            -0.0078      0.103     -0.076      0.940      -0.212       0.196
ln_gold.L4            -0.0851      0.107     -0.795      0.429      -0.298       0.128
ln_gold.L5             0.2285      0.092      2.471      0.015       0.045       0.412
fed_rate.L0            0.7242      4.607      0.157      0.875      -8.433       9.882
fed_rate.L1            8.7731      8.253      1.063      0.291      -7.634      25.180
fed_rate.L2          -15.0317      8.144     -1.846      0.068     -31.222       1.159
fed_rate.L3           -2.9135      7.644     -0.381      0.704     -18.110      12.283
fed_rate.L4           -3.5997      7.837     -0.459      0.647     -19.179      11.980
fed_rate.L5           19.9843      7.554      2.646      0.010       4.968      35.000
fed_rate.L6          -14.0385      4.527     -3.101      0.003     -23.037      -5.040
ln_dollar_index.L0    -1.9355      0.298     -6.504      0.000      -2.527      -1.344
ln_dollar_index.L1     0.7129      0.374      1.904      0.060      -0.031       1.457
vix.L0                 8.4251      2.396      3.517      0.001       3.663      13.188
vix.L1                -1.9381      2.282     -0.849      0.398      -6.475       2.599
ln_labor_claims.L0    -0.1029      0.148     -0.696      0.489      -0.397       0.191
ln_labor_claims.L1     0.0681      0.167      0.407      0.685      -0.264       0.401
ln_labor_claims.L2     0.0414      0.166      0.250      0.803      -0.288       0.371
ln_labor_claims.L3     0.0445      0.164      0.272      0.787      -0.281       0.370
ln_labor_claims.L4    -0.0531      0.165     -0.321      0.749      -0.382       0.276
ln_labor_claims.L5    -0.5017      0.150     -3.336      0.001      -0.801      -0.203
ln_labor_claims.L6     0.1685      0.138      1.223      0.225      -0.105       0.442
ln_ip.L0               2.0517      0.876      2.343      0.021       0.311       3.792
ln_ip.L1              -1.6792      0.913     -1.839      0.069      -3.494       0.136
ln_ppi.L0             -1.8781      0.591     -3.179      0.002      -3.052      -0.704
ln_ppi.L1              1.3878      0.725      1.915      0.059      -0.053       2.829
ln_ppi.L2             -0.7824      0.552     -1.418      0.160      -1.879       0.314
======================================================================================
"""

In [24]:
model_uecm = UECM(
    data_ardl[y_col], 
    lags=6, 
    exog=data_ardl[X_cols], 
    order=exog_order
)
res_uecm = model_uecm.fit()

# 2. รัน Bounds Test จากผลลัพธ์ของ UECM
# case 3 คือมี intercept แต่ไม่มี trend (นิยมใช้ที่สุด)
bt_results = res_uecm.bounds_test(case=3)

print("--- ARDL Bounds Test Results ---")
print(bt_results)

# 3. ดูค่า ECT (ในตาราง summary จะชื่อประมาณ 'diff.ln_gold.L1' หรือตัวแปรที่เป็นระดับ Level)
# หรือดูค่า Adjustment Term โดยตรง
print("\n--- UECM Summary (ดูค่า ECT และนัยสำคัญ) ---")
print(res_uecm.summary())

--- ARDL Bounds Test Results ---
BoundsTestResult
Stat: 3.04199
Upper P-value: 0.119
Lower P-value: 0.00793
Null: No Cointegration
Alternative: Possible Cointegration


--- UECM Summary (ดูค่า ECT และนัยสำคัญ) ---
                                  UECM Model Results                                 
Dep. Variable:                     D.ln_gold   No. Observations:                  120
Model:             UECM(6, 6, 1, 1, 6, 1, 2)   Log Likelihood                 215.218
Method:                      Conditional MLE   S.D. of innovations              7.028
Date:                       Thu, 19 Feb 2026   AIC                           -368.436
Time:                               14:49:26   BIC                           -283.614
Sample:                           07-31-2006   HQIC                          -334.012
                                - 12-31-2015                                         
                           coef    std err          z      P>|z|      [0.025      0.975]
---------

## ARDL-ECM Forcast

In [25]:
exog_order_pure = {}
for i in exog_order:
    exog_order_pure[i] = [int(j) for j in range(1, exog_order[i]+1)]
ar_order = ar_lag

train_data = df_test_ardl.copy()
test_data = df_ardl[df_ardl.index > train_date_str].copy()

history = train_data.copy()
predictions = []
actuals = test_data[y_col].values

print(f"Train Period: {train_data.index[0].date()} to {train_data.index[-1].date()} (Count: {len(train_data)})")
print(f"Test Period:  {test_data.index[0].date()} to {test_data.index[-1].date()} (Count: {len(test_data)})")
print(f"\nStarting Walk-Forward Forecast (OOS)")

for t in range(len(test_data)):
    model = ARDL(
        endog=history[y_col],
        lags=ar_order,
        exog=history[X_cols],
        order=exog_order_pure,
        trend='c'
    )
    model_fit = model.fit()
    
    next_exog = test_data.iloc[[t]][X_cols]
    
    pred = model_fit.predict(start=len(history), end=len(history), exog_oos=next_exog)
    yhat = pred.values[0]
    predictions.append(yhat)
    
    history = pd.concat([history, test_data.iloc[[t]]])
    
    # warking forward
    # history = history.iloc[1:]
    
    if (t+1) % 12 == 0:
        print(f"Step {t+1}: {test_data.index[t].date()} -> Pred={np.exp(yhat):.4f} | Actual={np.exp(actuals[t]):.4f}")

final_model = ARDL(endog=history[y_col], lags=ar_order, exog=history[X_cols], order=exog_order_pure, trend='c')
final_model_fit = final_model.fit()
next_exog_future = history.iloc[[-1]][X_cols]
pred_future = final_model_fit.predict(start=len(history), end=len(history), exog_oos=next_exog_future)
yhat_future = pred_future.values[0]
# predictions.append(yhat_future)

actual_price = np.exp(actuals)
pred_price = np.exp(predictions)

results = pd.DataFrame({
    'Actual': actuals,
    'Predicted' : predictions,
    'Error' : actuals - predictions,
    'Actual_Price': actual_price,
    'Predicted_Price': pred_price,
    'Error_Price' : actual_price - pred_price
}, index=test_data.index)
last_date = results.index[-1]
next_date = last_date + BusinessDay(n=1)
future_row = pd.DataFrame({
    'Actual': [np.nan],              
    'Predicted': [yhat_future],      
    'Error': [np.nan],               
    'Actual_Price': [np.nan],        
    'Predicted_Price': [np.exp(yhat_future)], 
    'Error_Price': [np.nan]  
}, index=[next_date]) 

results = pd.concat([results, future_row])       
results.round(2)

Train Period: 2006-01-31 to 2015-12-31 (Count: 120)
Test Period:  2016-01-31 to 2026-01-31 (Count: 121)

Starting Walk-Forward Forecast (OOS)
Step 12: 2016-12-31 -> Pred=1191.1052 | Actual=1150.0000
Step 24: 2017-12-31 -> Pred=1254.7065 | Actual=1306.3000
Step 36: 2018-12-31 -> Pred=1212.4400 | Actual=1278.3000
Step 48: 2019-12-31 -> Pred=1535.8826 | Actual=1519.5000
Step 60: 2020-12-31 -> Pred=1795.3367 | Actual=1893.1000
Step 72: 2021-12-31 -> Pred=1795.2158 | Actual=1827.5000
Step 84: 2022-12-31 -> Pred=1721.4329 | Actual=1819.7000
Step 96: 2023-12-31 -> Pred=2027.7798 | Actual=2062.3999
Step 108: 2024-12-31 -> Pred=2757.6705 | Actual=2629.2000
Step 120: 2025-12-31 -> Pred=4255.7622 | Actual=4325.6001


,Actual,Predicted,Error,Actual_Price,Predicted_Price,Error_Price
2016-01-31,7.02,6.97,0.05,1116.4,1060.36,56.04
2016-02-29,7.12,7.02,0.10,1233.9,1116.93,116.97
2016-03-31,7.12,7.13,-0.01,1234.2,1249.16,-14.96
2016-04-30,7.16,7.11,0.05,1289.2,1223.65,65.55
2016-05-31,7.10,7.16,-0.06,1214.8,1291.96,-77.16
...,...,...,...,...,...,...
2025-10-31,8.29,8.26,0.03,3982.2,3858.95,123.25
2025-11-30,8.35,8.30,0.05,4218.3,4031.58,186.72
2025-12-31,8.37,8.36,0.02,4325.6,4255.76,69.84
2026-01-31,8.46,8.40,0.06,4713.9,4444.58,269.32


In [26]:
rmse = np.sqrt(mean_squared_error(actual_price, pred_price))
mae = mean_absolute_error(actual_price, pred_price)

print(f"RMSE (USD): {rmse:.2f}")
print(f"MAE (USD):  {mae:.2f}")

RMSE (USD): 162.87
MAE (USD):  86.78


In [27]:
# plt.figure(figsize=(14, 7))

# plt.axvline(x=pd.to_datetime('2015-12-31'), color='gray', linestyle=':', label='Train/Test Split')

# plt.plot(df_ardl.index, np.exp(df_ardl[y_col]), label='Actual History', color='lightgray')
# plt.plot(test_data.index, actual_price, label='Actual Test Data', color='#1f77b4', linewidth=2)
# plt.plot(test_data.index, pred_price, label='Forecast (Pure OOS)', color='#d62728', linestyle='--', linewidth=2)

# plt.title('Gold Price Forecast: Out-of-Sample Testing (2016-Present)')
# plt.xlabel('Date')
# plt.ylabel('Price (USD)')
# plt.legend()
# plt.grid(True, alpha=0.3)
# plt.show()

In [28]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_ardl.index, 
    y=np.exp(df_ardl[y_col]),
    mode='lines',
    name='Actual History',
    line=dict(color='lightgray')
))

fig.add_trace(go.Scatter(
    x=test_data.index, 
    y=actual_price,
    mode='lines',
    name='Actual Test (2016-Present)',
    line=dict(color='#1f77b4', width=2)
))

fig.add_trace(go.Scatter(
    x=test_data.index, 
    y=pred_price,
    mode='lines',
    name='Forecast',
    line=dict(color='#d62728', width=2, dash='dash')
))

fig.update_layout(
    width=1000,
    height=700,
    autosize=False,
    title='Gold Price Forecast: Interactive Walk-Forward Validation',
    yaxis_title='Price (USD)',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(count=5, label="5y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(visible=True),
        type="date"
    ),
    template="plotly_white",
    legend=dict(x=0, y=1)
)

fig.show()

## Technical Data

In [29]:
def denoise_data(data, wavelet='db4', level=2):
    coeff = pywt.wavedec(data, wavelet, mode="per")
    sigma = (1/0.6745) * np.median(np.abs(coeff[-1] - np.median(coeff[-1])))
    uthesh = sigma * np.sqrt(2 * np.log(len(data)))
    new_coeff = [coeff[0]]
    for i in coeff[1:]:
        new_coeff.append(pywt.threshold(i, value=uthesh, mode='soft'))
    reconstructed = pywt.waverec(new_coeff, wavelet, mode='per')
    return reconstructed[:len(data)]

In [30]:
macro_feature = results[['Predicted']].copy()
macro_feature.columns = ['Macro_Signal']

# Resample monthly to daily and forward fill
macro_daily = macro_feature.resample('D').asfreq()
macro_daily = macro_daily.ffill()

# Normalize datetime index (remove time component)
macro_daily.index = pd.to_datetime(macro_daily.index.date)
macro_daily

,Macro_Signal
2016-01-31,6.966360
2016-02-01,6.966360
2016-02-02,6.966360
2016-02-03,6.966360
2016-02-04,6.966360
...,...
2026-01-29,8.356029
2026-01-30,8.356029
2026-01-31,8.399441
2026-02-01,8.399441


In [31]:
df_daily = pd.read_csv('all_gold_data.csv', index_col=0, parse_dates=True)
df_daily.sort_index(inplace=True)
df_daily = df_daily[~df_daily.index.duplicated(keep='first')]

# Normalize datetime index (remove time component)
df_daily.index = pd.to_datetime(df_daily.index.date)

df_daily['actual_close'] = df_daily['close'].copy()
df_daily['ln_close'] = np.log(df_daily['close'])

for col in ['open', 'high', 'low', 'close']:
    df_daily[col] = denoise_data(df_daily[col].values)

# Momentum & Trend
df_daily.ta.rsi(length=14, append=True)
df_daily.ta.macd(fast=12, slow=26, signal=9, append=True)
df_daily.ta.adx(length=14, append=True)
df_daily.ta.cci(length=20, append=True)

# Volatility & Bands
df_daily.ta.bbands(length=20, std=2, append=True)
df_daily.ta.atr(length=14, append=True)

# Moving Average Distances
df_daily.ta.ema(length=50, append=True)
df_daily.ta.ema(length=200, append=True)
df_daily['dist_ema50'] = (df_daily['close'] - df_daily['EMA_50']) / df_daily['EMA_50']
df_daily['dist_ema200'] = (df_daily['close'] - df_daily['EMA_200']) / df_daily['EMA_200']

# Statistical & Others
df_daily['daily_range'] = (df_daily['high'] - df_daily['low']) / df_daily['open']
rolling_mean = df_daily['close'].rolling(window=20).mean()
rolling_std = df_daily['close'].rolling(window=20).std()
df_daily['z_score'] = (df_daily['close'] - rolling_mean) / rolling_std

cols_to_drop = ['EMA_50', 'EMA_200', 'BBU_20_2.0', 'BBL_20_2.0', 'BBM_20_2.0']
df_daily.drop(columns=[c for c in cols_to_drop if c in df_daily.columns], inplace=True)
df_daily.dropna(inplace=True)
df_daily

,open,high,low,close,volume,actual_close,ln_close,RSI_14,MACD_12_26_9,MACDh_12_26_9,...,DMP_14,DMN_14,CCI_20_0.015,BBB_20_2.0,BBP_20_2.0,ATRr_14,dist_ema50,dist_ema200,daily_range,z_score
2006-10-17,590.204470,592.309480,586.468188,588.004651,14,589.700012,6.379614,8.986734,-6.402138,0.586069,...,8.694665,21.762750,-52.489688,3.382650,0.250696,4.312874,-0.028313,-0.025855,0.009897,-0.971965
2006-10-18,590.428753,592.107271,587.275108,588.776774,18,588.900024,6.378256,14.883959,-6.049307,0.751120,...,8.004774,20.035952,-40.883658,3.097427,0.306981,4.349966,-0.026005,-0.024338,0.008184,-0.752526
2006-10-19,590.739230,591.881702,587.885553,589.701432,103,599.000000,6.395262,21.448128,-5.630172,0.936204,...,7.476441,18.713535,-29.378781,2.802161,0.376617,4.324693,-0.023538,-0.022584,0.006765,-0.481035
2006-10-20,590.990468,591.066774,588.490627,590.917586,4,593.000000,6.385194,29.183715,-5.140613,1.140610,...,7.148867,17.893618,-18.880856,2.522533,0.473645,4.199797,-0.020697,-0.020367,0.004359,-0.102750
2006-10-23,591.715386,591.376804,589.593221,592.189806,4,579.700012,6.362511,36.255626,-4.596986,1.347390,...,7.472601,17.327562,3.732675,2.270539,0.589057,4.027210,-0.017873,-0.018080,0.003014,0.347209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-02-12,5013.759701,5052.848650,4892.299705,4917.772610,484,4923.700195,8.501816,55.545082,112.228378,-16.800812,...,26.521351,26.779879,19.022103,14.291540,0.514799,167.919639,0.062376,0.246299,0.032022,0.057698
2026-02-13,4948.753051,5032.788054,4918.907896,4979.014416,232,5022.000000,8.521584,58.076527,108.146171,-16.706415,...,25.193934,25.439523,23.793906,13.325503,0.582725,164.140768,0.072426,0.258541,0.023012,0.322521
2026-02-17,4969.809466,4985.288673,4839.350022,4898.634865,540,4882.899902,8.493495,53.750277,97.303382,-22.039363,...,23.581153,27.300755,-23.847272,12.153355,0.434370,162.840617,0.052838,0.235295,0.029365,-0.255874
2026-02-18,4866.079711,4973.171967,4857.081570,4945.706285,540,4986.500000,8.514490,55.825565,91.454434,-22.310649,...,22.355213,25.881440,-21.463828,11.587703,0.495888,159.501315,0.060337,0.244105,0.023857,-0.016031


In [32]:
"""
For joining daily-marco variable to technical data
"""

macro_to_merge = dollar_index.join(vix, how='inner')
macro_to_merge = macro_to_merge[macro_to_merge.index >= '2016-02-01']
macro_to_merge

,Dollar Index,VIX
2016-02-01,115.3331,0.012586
2016-02-02,115.5530,0.013846
2016-02-03,114.6144,0.013638
2016-02-04,113.6012,0.013758
2016-02-05,114.2935,0.014728
...,...,...
2026-02-09,117.6392,0.010936
2026-02-10,117.5216,0.011207
2026-02-11,117.4601,0.011118
2026-02-12,117.5376,0.013115


In [33]:
df_final = macro_daily.join(df_daily, how='right')
df_final = df_final.join(macro_to_merge, how='left')
df_final = df_final.ffill()
df_final = df_final.dropna()
forecast_horizon = 5
for i in range(1, forecast_horizon + 1):
    col_name = f'target_return_{i}d'
    df_final[col_name] = np.log(df_final['actual_close']).shift(-i) - np.log(df_final['close'])

df_predict_latest = df_final.tail(forecast_horizon).copy()

# threshold = 0.00
# choices = [1, 0]
# for i in range(1, forecast_horizon + 1):
#     target_col = f'target_return_{i}d'
#     signal_col = f'signal_{i}d'
    
#     conditions = [
#         (df_final[target_col] >= threshold),
#         (df_final[target_col] < -threshold)
#     ]
    
#     df_final[signal_col] = np.select(conditions, choices, default=0)


df_final.to_csv('gold_technical.csv', index=True)
df_final

,Macro_Signal,open,high,low,close,volume,actual_close,ln_close,RSI_14,MACD_12_26_9,...,dist_ema200,daily_range,z_score,Dollar Index,VIX,target_return_1d,target_return_2d,target_return_3d,target_return_4d,target_return_5d
2016-02-01,6.966360,1138.312113,1145.745089,1130.798083,1142.176701,1682,1127.900024,7.028113,94.787598,12.901301,...,0.007635,0.013131,1.766747,115.3331,0.012586,-0.013110,-0.000768,0.013413,0.013586,0.047634
2016-02-02,6.966360,1142.965365,1150.809821,1134.631584,1146.201707,1230,1127.300049,7.027581,95.306617,13.796242,...,0.011073,0.014155,1.743814,115.5530,0.013846,-0.004286,0.009895,0.010068,0.044116,0.044784
2016-02-03,6.966360,1147.840679,1156.231122,1138.550980,1150.242315,1630,1141.300049,7.039923,95.762752,14.662512,...,0.014490,0.015403,1.723755,114.6144,0.013638,0.006376,0.006549,0.040597,0.041265,0.037922
2016-02-04,6.966360,1153.603724,1162.730081,1144.273492,1156.205971,771,1157.599976,7.054104,96.329719,15.649853,...,0.019549,0.015999,1.778243,113.6012,0.013758,0.001378,0.035426,0.036094,0.032751,0.076318
2016-02-05,6.966360,1160.333231,1170.389159,1151.967138,1164.262496,877,1157.800049,7.054277,96.927781,16.887750,...,0.026381,0.015876,1.886122,114.2935,0.014728,0.028482,0.029150,0.025807,0.069374,0.062297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-02-12,8.474213,5013.759701,5052.848650,4892.299705,4917.772610,484,4923.700195,8.501816,55.545082,112.228378,...,0.246299,0.032022,0.057698,117.5376,0.013115,0.020973,-0.007116,0.013879,0.023617,NaN
2026-02-13,8.474213,4948.753051,5032.788054,4918.907896,4979.014416,232,5022.000000,8.521584,58.076527,108.146171,...,0.258541,0.023012,0.322521,117.5258,0.012977,-0.019493,0.001502,0.011241,NaN,NaN
2026-02-17,8.474213,4969.809466,4985.288673,4839.350022,4898.634865,540,4882.899902,8.493495,53.750277,97.303382,...,0.235295,0.029365,-0.255874,117.5258,0.012977,0.017778,0.027517,NaN,NaN,NaN
2026-02-18,8.474213,4866.079711,4973.171967,4857.081570,4945.706285,540,4986.500000,8.514490,55.825565,91.454434,...,0.244105,0.023857,-0.016031,117.5258,0.012977,0.017953,NaN,NaN,NaN,NaN


In [34]:
df_final.columns

Index(['Macro_Signal', 'open', 'high', 'low', 'close', 'volume',
       'actual_close', 'ln_close', 'RSI_14', 'MACD_12_26_9', 'MACDh_12_26_9',
       'MACDs_12_26_9', 'ADX_14', 'DMP_14', 'DMN_14', 'CCI_20_0.015',
       'BBB_20_2.0', 'BBP_20_2.0', 'ATRr_14', 'dist_ema50', 'dist_ema200',
       'daily_range', 'z_score', 'Dollar Index', 'VIX', 'target_return_1d',
       'target_return_2d', 'target_return_3d', 'target_return_4d',
       'target_return_5d'],
      dtype='object')

## CNN-LSTM

In [56]:
# --- Config ---
N_MODELS = 5          # Number of models in the Ensemble team
WINDOW_SIZE = 30      # Lookback window
BATCH_SIZE = 8        # Training batch size

# Set Seed for reproducible results
def set_seeds(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
set_seeds(42)

print("🚀 Loading Data...")

# --- 1. Data Preparation ---
# Load full dataset. This contains the latest days even if targets are NaN
df = pd.read_csv('gold_technical.csv', index_col=0, parse_dates=True)

df = df[:-5]

# Create training set by dropping rows with NaN targets
df_train_full = df.dropna(subset=['target_return_5d']).copy()

feature_cols = [c for c in df.columns if 'target' not in c and c != 'actual_close' and c != 'close']
target_cols = ['target_return_1d', 'target_return_2d', 'target_return_3d', 'target_return_4d', 'target_return_5d']

X = df_train_full[feature_cols].values
y = df_train_full[target_cols].values

# Split Train/Test (80/20)
train_size = int(len(X) * 0.8)
X_train_raw, X_test_raw = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Scaling
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train_raw)
X_test_scaled = scaler_X.transform(X_test_raw)

def create_sequences(X, y, window_size):
    Xs, ys = [], []
    for i in range(len(X) - window_size):
        Xs.append(X[i:(i + window_size)])
        ys.append(y[i + window_size])
    return np.array(Xs), np.array(ys)

X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train, WINDOW_SIZE)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test, WINDOW_SIZE)

print(f"Training Shape: {X_train_seq.shape}, Target Shape: {y_train_seq.shape}")

# --- 2. Ensemble Training Loop ---
print(f"\n🚀 Starting Ensemble System ({N_MODELS} Models)...")
model_files = [] 

for i in range(N_MODELS):
    print(f"\nTraining Model {i+1}/{N_MODELS} ...")
    
    set_seeds(42 + i)
    
    model = Sequential([
        Conv1D(filters=64, kernel_size=2, activation='swish', input_shape=(WINDOW_SIZE, len(feature_cols))),
        MaxPooling1D(pool_size=2),
        Bidirectional(LSTM(128, return_sequences=False, activation='tanh')),
        Dropout(0.3),
        Dense(64, activation='swish'),
        Dense(5)
    ])

    model.compile(optimizer=Adam(learning_rate=0.001), loss=Huber(), metrics=['mae'])
    # model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    
    filename = f'gold_ensemble_{i}.keras'
    model_files.append(filename)
    
    callbacks = [
        ModelCheckpoint(filename, save_best_only=True, monitor='val_loss', mode='min', verbose=0),
        EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=0),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=0)
    ]

    model.fit(
        X_train_seq, y_train_seq,
        epochs=100, 
        batch_size=BATCH_SIZE,
        validation_data=(X_test_seq, y_test_seq),
        callbacks=callbacks,
        verbose=0
    )
    print(f"✅ Model {i+1} Saved.")

# --- 3. Real-world Inference (Predicting from the absolute latest date) ---
print(f"\n🔮 Predicting the next 5 days (from the average of {N_MODELS} models)...")

# Use the full df to extract the latest 30 days
last_window = df[feature_cols].tail(WINDOW_SIZE).values
last_window_scaled = scaler_X.transform(last_window).reshape(1, WINDOW_SIZE, len(feature_cols))

# Predict with all models
all_predictions = []
for filename in model_files:
    model = tf.keras.models.load_model(filename)
    pred = model.predict(last_window_scaled, verbose=0)[0]
    all_predictions.append(pred)

# Average the predictions
avg_prediction_returns = np.mean(all_predictions, axis=0)

current_actual_price = df['actual_close'].iloc[-1]
last_date = df.index[-1]

# Convert log returns back to target prices
predicted_prices = [current_actual_price * np.exp(ret) for ret in avg_prediction_returns]

🚀 Loading Data...
Training Shape: (1987, 30, 23), Target Shape: (1987, 5)

🚀 Starting Ensemble System (5 Models)...

Training Model 1/5 ...
✅ Model 1 Saved.

Training Model 2/5 ...
✅ Model 2 Saved.

Training Model 3/5 ...
✅ Model 3 Saved.

Training Model 4/5 ...
✅ Model 4 Saved.

Training Model 5/5 ...
✅ Model 5 Saved.

🔮 Predicting the next 5 days (from the average of 5 models)...


In [57]:
market_cal = mcal.get_calendar('CMEGlobex_Gold')
start_search = last_date + pd.Timedelta(days=1)
end_search = start_search + pd.Timedelta(days=15) # Search ahead for 15 days

# Extract valid trading days and select the first 5 (remove timezone)
valid_days = market_cal.valid_days(start_date=start_search, end_date=end_search)
next_5_trading_days = valid_days[:5].tz_localize(None)

print("\n" + "="*55)
print(f"📅 Latest Closing Price ({last_date.strftime('%Y-%m-%d')}): {current_actual_price:.2f}")
print(f"🏆 Ensemble Forecast (CME Gold Calendar):")
print("-" * 55)

for i, (price, date) in enumerate(zip(predicted_prices, next_5_trading_days), 1):
    trend = "🟢 UP" if price > current_actual_price else "🔴 DOWN"
    diff = price - current_actual_price
    print(f"Day {i} ({date.strftime('%Y-%m-%d')}): {price:.2f}  {trend} ({diff:+.2f})")
print("="*55)


📅 Latest Closing Price (2026-02-11): 5071.60
🏆 Ensemble Forecast (CME Gold Calendar):
-------------------------------------------------------
Day 1 (2026-02-12): 5010.33  🔴 DOWN (-61.27)
Day 2 (2026-02-13): 5230.52  🟢 UP (+158.92)
Day 3 (2026-02-16): 5061.07  🔴 DOWN (-10.53)
Day 4 (2026-02-17): 5028.25  🔴 DOWN (-43.35)
Day 5 (2026-02-18): 5046.78  🔴 DOWN (-24.82)


In [58]:
print(f"\n📊 Evaluating historical RMSE on Test Set...")

test_preds_all = []
for filename in model_files:
    model = tf.keras.models.load_model(filename)
    pred = model.predict(X_test_seq, verbose=0) 
    test_preds_all.append(pred)

y_pred_test_avg = np.mean(test_preds_all, axis=0) 

test_start_index = train_size + WINDOW_SIZE
base_prices_test = df_train_full['close'].iloc[test_start_index:].values

min_len = min(len(y_pred_test_avg), len(base_prices_test))
y_pred_clipped = y_pred_test_avg[:min_len]
y_test_clipped = y_test[:min_len] 
base_prices_clipped = base_prices_test[:min_len].reshape(-1, 1)

all_actual_prices = []
all_pred_prices = []

print("\n" + "="*50)
print(f"📊 Accuracy Summary (Ensemble RMSE Price Basis)")
print("="*50)

for i in range(5):
    true_price = base_prices_clipped * np.exp(y_test_clipped[:, i].reshape(-1, 1))
    pred_price = base_prices_clipped * np.exp(y_pred_clipped[:, i].reshape(-1, 1))

    all_actual_prices.append(true_price)
    all_pred_prices.append(pred_price)

    rmse = np.sqrt(mean_squared_error(true_price, pred_price))
    print(f"RMSE {i+1}d : ${rmse:.2f}")

flat_actual = np.concatenate(all_actual_prices).flatten()
flat_pred = np.concatenate(all_pred_prices).flatten()
combined_rmse = np.sqrt(mean_squared_error(flat_actual, flat_pred))

print("-" * 50)
print(f"📉 Combined RMSE (1-5d): ${combined_rmse:.2f}")
print("="*50)


📊 Evaluating historical RMSE on Test Set...

📊 Accuracy Summary (Ensemble RMSE Price Basis)
RMSE 1d : $45.08
RMSE 2d : $61.28
RMSE 3d : $67.62
RMSE 4d : $66.31
RMSE 5d : $108.47
--------------------------------------------------
📉 Combined RMSE (1-5d): $72.83
